<a href="https://colab.research.google.com/github/Vikas89-DS/qlora-fine-tuning-project/blob/main/Fine_Tuning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch transformers datasets accelerate peft bitsandbytes trl pandas openpyxl streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.4/517.4 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.0 MB/s eta 0:00:00


In [2]:
import os
os.environ["ACCELERATE_MIXED_PRECISION"] = "no"

In [3]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

In [4]:
# 1. Data Loading
data = pd.read_csv(r"/content/drive/MyDrive/drugs_side_effects_drugs_com.csv")

In [5]:
data.head(20)

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,related_drugs,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod...",87%,Rx,D,N,X,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,6.8,760.0,https://www.drugs.com/doxycycline.html,https://www.drugs.com/condition/acne.html
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir",82%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.2,449.0,https://www.drugs.com/spironolactone.html,https://www.drugs.com/condition/acne.html
2,minocycline,Acne,"skin rash, fever, swollen glands, flu-like sym...",minocycline,Tetracyclines,"Dynacin, Minocin, Minolira, Solodyn, Ximino, V...",48%,Rx,D,N,NaN,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,5.7,482.0,https://www.drugs.com/minocycline.html,https://www.drugs.com/condition/acne.html
3,Accutane,Acne,problems with your vision or hearing; muscle o...,isotretinoin (oral),"Miscellaneous antineoplastics, Miscellaneous u...",NaN,41%,Rx,X,N,X,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.9,623.0,https://www.drugs.com/accutane.html,https://www.drugs.com/condition/acne.html
4,clindamycin,Acne,hives ; difficult breathing; swelling of your ...,clindamycin topical,"Topical acne agents, Vaginal anti-infectives","Cleocin T, Clindacin ETZ, Clindacin P, Clindag...",39%,Rx,B,N,NaN,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.4,146.0,https://www.drugs.com/mtm/clindamycin-topical....,https://www.drugs.com/condition/acne.html
5,Aldactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...",CaroSpir,35%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.6,8.0,https://www.drugs.com/aldactone.html,https://www.drugs.com/condition/acne.html
6,tretinoin,Acne,hives ; difficult breathing; swelling of your ...,tretinoin topical,Topical acne agents,"Altreno, Atralin, Avita, Refissa, Renova, Reti...",30%,Rx,C,N,NaN,NaN,Acne Other names: Acne Vulgaris; Blackheads; B...,7.7,439.0,https://www.drugs.com/mtm/tretinoin-topical.html,https://www.drugs.com/condition/acne.html
7,isotretinoin,Acne,problems with your vision or hearing; muscle o...,isotretinoin (oral),"Miscellaneous antineoplastics, Miscellaneous u...","Absorica, Accutane, Amnesteem, Claravis, Myori...",26%,Rx,X,N,X,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,8.0,999.0,https://www.drugs.com/mtm/isotretinoin.html,https://www.drugs.com/condition/acne.html
8,Bactrim,Acne,"skin rash, fever, swollen glands, joint pain, ...",sulfamethoxazole and trimethoprim,Sulfonamides,Bactrim DS,20%,Rx,D,N,X,NaN,Acne Other names: Acne Vulgaris; Blackheads; B...,8.5,96.0,https://www.drugs.com/bactrim.html,https://www.drugs.com/condition/acne.html
9,Retin-A,Acne,hives; difficult breathing; swelling of your f...,Retin-A,Topical acne agents,NaN,17%,Rx,C,N,NaN,NaN,Acne Other names: Acne Vulgaris; Blackheads; B...,7.9,86.0,https://www.drugs.com/retin-a.html,https://www.drugs.com/condition/acne.html


In [6]:
# 2. check data for duplicates and null values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2931 entries, 0 to 2930
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   drug_name                      2931 non-null   object 
 1   medical_condition              2931 non-null   object 
 2   side_effects                   2807 non-null   object 
 3   generic_name                   2888 non-null   object 
 4   drug_classes                   2849 non-null   object 
 5   brand_names                    1718 non-null   object 
 6   activity                       2931 non-null   object 
 7   rx_otc                         2930 non-null   object 
 8   pregnancy_category             2702 non-null   object 
 9   csa                            2931 non-null   object 
 10  alcohol                        1377 non-null   object 
 11  related_drugs                  1462 non-null   object 
 12  medical_condition_description  2931 non-null   o

In [7]:
# 3. Data Cleaning
data = data.dropna(subset=["side_effects"]).reset_index(drop=True)
print(f"Clean rows after drop: {len(data)}")

Clean rows after drop: 2807


In [8]:
# 4. Prompt Formatting
def create_prompt(row):
    return f"""### Drug: {row['drug_name']}
### Used for: {row['medical_condition']}

Provide complete side effects of the drug:

### Side Effects:
"""

In [9]:
#5. Text column creation
data["prompt"] = data.apply(create_prompt, axis=1)

In [10]:
#6. Keep only required column
data = data[["prompt", "side_effects"]]

In [11]:
# 7. Dataset Splitting
hf_dataset = Dataset.from_pandas(data).train_test_split(test_size=0.1)
print(f"Train: {len(hf_dataset['train'])}, Test: {len(hf_dataset['test'])}")

Train: 2526, Test: 281


In [12]:
#8. Model & Tokenizer setup
model_name = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [13]:
#9. Final text field creation
def format_text(example):
    return {
        "text": example["prompt"] + str(example["side_effects"]) + tokenizer.eos_token
    }

hf_dataset = hf_dataset.map(format_text, remove_columns=["prompt", "side_effects"])

Map:   0%|          | 0/2526 [00:00<?, ? examples/s]

Map:   0%|          | 0/281 [00:00<?, ? examples/s]

In [14]:
#10. 4-bit Quantization Config (QLoRA)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [15]:
#11. Load Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [16]:
#12. Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False

In [17]:
#13. LoRA Config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [18]:
#14. Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,145,728 || all params: 3,824,225,280 || trainable%: 0.0823


In [19]:
#15. Enable Gradient Checkpointing
model.gradient_checkpointing_enable()

In [20]:
# 16. Training Config
sft_config = SFTConfig(
    output_dir="./drug_side_effects_bot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    fp16=False,
    bf16=False,
    max_grad_norm=0.0,
    optim="adamw_torch",
    report_to="none",
    warmup_steps=50,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    max_length=1024,
    packing=False,
    dataset_text_field="text"
)

In [21]:
# 17. Trainer Setup
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=hf_dataset["train"],
    eval_dataset=hf_dataset["test"],
    peft_config=lora_config,
)

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/2526 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2526 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (12193 > 4096). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/2526 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/281 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/281 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/281 [00:00<?, ? examples/s]

In [ ]:
# 18. Start Training
trainer.train()

Step,Training Loss
10,1.777500
